# Positioning With Doppler Observables

In [27]:
# Section 1 - Imports

import datetime
import attotime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math
from os.path import join

from dsoclasses.doris.algorithms import beacon_nominal_frequency
from dsoclasses.rinex.doris.rinex import DorisRinex
from dsoclasses.time.pyattotime import at2pt, fsec2asec
from dsoclasses.orbits import sp3c, interpolator
from dsoclasses.geodesy import transformations
from dsoclasses.gnss import systems as gs
from dsoclasses.gnss import algorithms as alg
from dsoclasses.troposphere.vmf3 import SiteVmf3
from dsoclasses.sinex import sinex

# Section 1.2 – Define File Paths
data_path = "/home/xanthos/Software/AcademicSoftware/data"
drinex = join(data_path, "s6arx24001.001")
dpod = join(data_path, "dpod2020_041.snx")
dpod_freq_corr = join(data_path, "dpod2020_041_freq_corr.txt")
dsp3 = join(data_path, "ssas6a20.b23357.e24001.DG_.sp3.001")
vmf3_data = join(data_path, "y2024.vmf3_d")

# Section 1.3 - Load Sp3 + Initialize Interpolator 
intrp = interpolator.Sp3Interpolator.from_sp3(dsp3, ['L'], interval_in_sec=310, min_data_pts=10, itype='Barycentric')
# the sp3 only holds one satellite, let's get its id:
assert len(list(intrp._interpolators.keys())) == 1
sp3_id = list(intrp._interpolators.keys())[0]

# Section 1.4 - Intitialize DorisRinex and select a beacon (Dionysos/DIOB)
rnx = DorisRinex(drinex)

# Section 1.5 - Choose a station and get coordinates (DIOB DORIS beacon)
site_name = 'DIOB'
diob_crd = sinex.extract_sinex_coordinates(dpod, [site_name], rnx.time_first_obs, True)[site_name]
rsta = np.array([diob_crd['X'], diob_crd['Y'], diob_crd['Z']])
lat, lon, hgt = transformations.car2ell(*rsta)

# Section 1.6 - Initialize VMF3 for chosen site
vmf = SiteVmf3(vmf3_data, [site_name])

# Section 1.7 - Constants
C = 299792458e0 # Speed of light, [m/sec]
CUTOFF_ANGLE = np.radians(7.) # Elevation cut-off angle

In [28]:
# Nominal Frequencies for chosen site
s1, u2 = beacon_nominal_frequency(rnx.kfactor(site_name))

In [40]:
# frequencies
feN = s1
frN, _ = beacon_nominal_frequency(0)

# Initialize variables for looping
# --------------------------------
last_tai   = None   # previous (to current) observation epoch in TAI
raw_doppler = []    # Doppler observations collected
pass_nr = 0         # satellite pass number

# for every block in the RINEX file
for block in rnx:
    # for every beacon in the block
    for beacon, data in block:
        # match DIOB
        if beacon == rnx.name2id(site_name):
            
            # use the block-provided clock correction to get to (approximate) TAI
            # date(TAI) = epoch + receiver clock offset
            tai = block.t() + attotime.attotimedelta(nanoseconds=block.clock_offset() * 1e9)

            # satellite position
            satx, saty, satz, _ = intrp.sat_at(sp3_id, tai)
            rsat = np.array([satx, saty, satz])

            # compute range, azimouth and elevation between site and satellite
            r, az, el = transformations.azele(rsat, rsta)

            # only consider observation if above cut-off angle
            if el < CUTOFF_ANGLE:
                # print(f'Observation at {at2pt(tai)} below cut-off angle ({np.degrees(el):.1f}); skipped!')
                continue

            # get the 2GHz observation
            cphase = data['L1']['value']

            # apply frequency offset - get true receiver frequency
            frT = frN * (1. + data['F']['value'] * 1e-11)

            # compute tropospheric delay at elevation
            dti = vmf.tropo_delay(site_name, lat, lon, el, at2pt(tai))

            # compute difference to previous obs, if any
            if last_tai is not None and (tai - last_tai).total_seconds() <= 10.:
                # time diff from previous observation
                dt = float((tai-last_tai).total_nanoseconds()) * 1e-9
                assert dt != 0e0
                # observed range-rate
                orr = (C/feN) * (feN-frT-(cphase-last_cphase)/dt)
                # 
                raw_doppler.append({
                    'rsat1': last_rsat,
                    'rsat2': rsat,
                    'el': last_el + (el-last_el)/2.,
                    'dt': dt,
                    'v_obs': orr,
                    'dT': (dti-last_dti)/dt,
                    'pass': pass_nr
                })
                
            # is this a different pass ?
            elif last_tai is not None and (tai - last_tai).total_seconds() > 100.:
                # print(f'New pass found! now # passes is {pass_nr}')
                pass_nr += 1

            # assign values for next computation
            last_tai = tai
            last_rsat = rsat
            last_el = el
            last_dti = dti
            last_cphase = cphase

print(f"Collected #{len(raw_doppler)} doppler obs, in {pass_nr} passes")

Collected #965 doppler obs, in 7 passes


In [45]:
from collections import Counter
import bisect

def remove_minor_passes(raw_dop, min_obs_in_pass, renumber=True):
    """
    Filter out passes with too few observations. Optionally re-number the remaining passes.

    raw_dop: list[dict], each dict has a key 'pass_nr' (int)
    min_obs_in_pass: int, minimum observations required to keep a pass
    renumber: bool, if True make pass_nr contiguous after removals (0..N-1 in ascending order)
    """
    # 1) Count observations per pass
    counts = Counter(d['pass'] for d in raw_dop)
    print('Number of observations per pass:')
    for k,v in counts.items(): print(f'Pass Nr.: {k} # Obs. {v:4d}')

    # 2) Identify passes to remove
    to_remove = {pn for pn, c in counts.items() if c < min_obs_in_pass}
    if not to_remove:
        # Nothing to remove; optionally renumber and return
        kept = [d.copy() for d in raw_dop]
        if renumber:
            remaining = sorted({d['pass'] for d in kept})
            mapping = {old: new for new, old in enumerate(remaining)}
            for d in kept:
                d['pass'] = mapping[d['pass']]
        return kept

    # 3) Keep only entries whose pass_nr is not removed
    kept = [d.copy() for d in raw_dop if d['pass'] not in to_remove]

    if not renumber:
        return kept

    # 4) Renumber: subtract how many removed pass numbers are < current pass_nr
    removed_sorted = sorted(to_remove)
    for d in kept:
        pn = d['pass']
        shift = bisect.bisect_left(removed_sorted, pn)
        d['pass'] = pn - shift
    return kept

raw_doppler = remove_minor_passes(raw_doppler, 70)
pass_nr = max({d['pass'] for d in raw_doppler}) + 1
print(f"Collected #{len(raw_doppler)} doppler obs, in {pass_nr} passes")

Number of observations per pass:
Pass Nr.: 0 # Obs.   87
Pass Nr.: 1 # Obs.   66
Pass Nr.: 2 # Obs.  198
Pass Nr.: 3 # Obs.  118
Pass Nr.: 4 # Obs.  142
Pass Nr.: 5 # Obs.  198
Pass Nr.: 6 # Obs.  152
Collected #895 doppler obs, in 6 passes
